SQL adalah salah satu bahasa populer untuk pemrosesan dan analisis data. Spark mendukung SQL untuk memproses DataFrame. Dalam latihan ini kita akan menggunakan spark SQL tanpa database.


In [ ]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00


In [ ]:
from pyspark.sql import SparkSession

Inisialisasi spark session untuk berinteraksi dengan Spark cluster.

In [ ]:
spark = SparkSession.builder.appName('DataFrame Basics').getOrCreate()

Download dataset

In [ ]:
!wget https://github.com/urfie/SparkSQL-dengan-Hive/raw/main/datasets/application_record_header.csv.gz

Load ke dataframe

In [ ]:
df = spark.read.csv("application_record_header.csv.gz", header=True, inferSchema=True)

Sebelum menggunakan SQL, kita perlu membuat temporary table dari dataframe yang akan kita olah.

Gunakan fungsi `createOrReplaceTempView(nama_tabel)` pada dataframe tersebut.

In [ ]:
df.createOrReplaceTempView("app_record")

Selanjutnya kita bisa menggunakan nama tabel yang sudah kita definisikan dalam SQL statement.

Untuk mengeksekusi SQL statement, kita gunakan fungsi `sql(sqlstatement)` pada spark session.

In [ ]:
spark.sql("select count(*) from app_record").show()

In [ ]:
spark.sql("select * from app_record limit 5").show()

Kita akan melakukan join salah satu kolom dengan data referensi dan melakukan agregasi. Sebelumnya kita buat dataframe referensi dan membuat temporary viewnya

In [ ]:
mydata = (
    ('Lower secondary',1),
    ('Secondary / secondary special',2),
    ('Academic degree',3),
    ('Incomplete higher',4),
    ('Higher education',5))

ref_edu = spark.createDataFrame(mydata).toDF("NAME_EDUCATION_TYPE", "EDU_LEVEL")
ref_edu.createOrReplaceTempView("ref_edu")
spark.sql("select * from ref_edu").show()

Kita lakukan join, agregat, kemudian kita simpan ke tabel

In [ ]:
spark.sql("""SELECT edu_level, count(1) as number_of_app FROM
              (SELECT ref_edu.EDU_LEVEL as edu_level
                FROM app_record LEFT JOIN ref_edu
                ON app_record.NAME_EDUCATION_TYPE=ref_edu.NAME_EDUCATION_TYPE)
             GROUP BY edu_level SORT BY edu_level""").write.saveAsTable(name="aggregated_edu", mode="overwrite")

Kita tampilkan hasilnya dengan menggunakan perintah `describe formatted`

In [ ]:
spark.sql("describe formatted aggregated_edu").show(truncate = False)

In [ ]:
%ls -l /content/spark-warehouse/aggregated_edu/